In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

import openai as client

# Retrieval: 애플리케이션에서 특정 데이터를 검색하고 사용할 수 있는 인터페이스를 제공합니다.
assistant = client.beta.assistants.create(
    name="Book Assistant",
    instructions="You help users with their question on the files they upload.",
    model="gpt-4o-mini",
    tools=[{"type": "file_search"}],
)
assistant

assistant_id = "asst_aHluTGWMYIsAeyxHl0sEsLxR"


ModuleNotFoundError: No module named 'openai'

In [2]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want you to help me with this file.",
        }
    ]
)
thread

Thread(id='thread_ceWYGFMs9jjtfEcwzHlK520A', created_at=1726379770, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [3]:
file = client.files.create(
    file = client.file_from_path("../files/chapter_one.txt"),
    purpose="assistants",
)
file

FileObject(id='file-RpelmJdy8wpkTokUjg9HHZFP', bytes=35420, created_at=1726379771, filename='chapter_one.txt', object='file', purpose='assistants', status='processed', status_details=None)

In [4]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content="",
    attachments=[
        {
            'file_id':file.id,
            'tools':[
                {
                    "type":"file_search",
                }
            ]
        }
    ]
)

NameError: name 'client' is not defined

In [5]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

In [6]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id,
    )
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(f"Source: {annotation.file_citation}")



def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        # print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs,
    )

In [7]:
get_run(run.id, thread.id).status


'in_progress'

In [10]:
get_messages(thread.id)


user: I want you to help me with this file.
user:  
assistant: It seems you haven't specified what you need help with regarding the file. Please provide more details about your request or the specific information you're looking for!


In [9]:
send_message(
    thread.id,
    "Where does he work?",
)

BadRequestError: Error code: 400 - {'error': {'message': "Can't add messages to thread_ceWYGFMs9jjtfEcwzHlK520A while a run run_EcqEtEzI4AQwHzBofFru8QoL is active.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [25]:
get_messages(thread.id)


user: I want you to help me with this file.
user:  
assistant: It seems like your message is empty. Could you please provide more details about how you would like me to assist you with the uploaded file?
user: Where does he work?
